# Klasterinė analizė: Pagrindinės sąvokos 
# *(Cluster analysis: Basic concepts)*

Klasterinė analizė priskiriama prie *("Unsupervised learning")* Mokymasis be mokytojo t.y. nėra iš anksto nustatytų klasių.

In [133]:
                                import os
                                import pyodbc 
                                import pandas                         as pd
                                import networkx                       as nx
                                import matplotlib.pyplot              as plt
                                import numpy                          as np
from sklearn                    import cluster
from sklearn                    import mixture
from collections                import defaultdict
from sklearn.metrics.cluster    import normalized_mutual_info_score
from sklearn.metrics.cluster    import adjusted_rand_score

In [115]:
# Comment this if the data visualisations doesn't work
%matplotlib inline
plt.style.use('bmh')

In [116]:
path = 'C:/Users/banarn/optimisation_shortest_path/'
os.chdir(path)

In [117]:
df = pd.read_csv(path + 'output/data_nm.csv')
df = df.set_index('Product')
df.head()

,NPC,Parts,Path,Width,Lenght,LiftPushSpeed,LiftRegularSpeed,RT1EntranceSpeed,RT1ExitSpeed,RT2EntranceSpeed,...,Position_3,Position_4,Position_5,BR_BR2,BR_BR3,Color_black,Color_chary,Color_none,Color_oack,Color_white
Product,,,,,,,,,,,,,,,,,,,,,
BR3-IKW202,-0.194706,-0.194969,1,0.099728,-0.054031,0.094939,-0.225517,0.071878,-0.084769,0.058467,...,1,0,0,0,1,0,0,0,0,1
BR3-ITW101-K,-0.028040,0.805031,0,0.043267,-0.165659,-0.143157,-0.082659,-0.166217,-0.084769,0.058467,...,0,0,0,0,1,0,0,0,0,1
BR3-ITW302,-0.111373,0.805031,0,0.174103,-0.138334,-0.000299,-0.082659,-0.023360,0.045666,0.169578,...,1,0,0,0,1,0,0,0,0,1
BR3-ITW301,-0.028040,0.805031,0,-0.016451,-0.138334,0.142558,0.203055,0.024259,0.306535,0.317727,...,1,0,0,0,1,0,0,0,0,1
BR3-ITW104,-0.111373,0.805031,0,0.203962,-0.138334,-0.143157,-0.368374,-0.166217,-0.084769,0.058467,...,0,0,0,0,1,0,0,0,0,1


methods for clustering:
    
* [HDBSCAN](http://hdbscan.readthedocs.io/en/latest/index.html)
* DBSCAN
* K Narest

## Požymių savybių atranka *(Feature selection)*

Parinkimas atliekamas remiantis **Exploratory analysis** rezulstatais

[Worcking with dictionarys](https://stackoverflow.com/a/8381589/7347438)

In [118]:
kClusters = 4
labels = list(df.index.values)
results = {'Product': labels}
nmiResults = []
arsResults = []

## Artumo matas *(Proximity measure)*


In [119]:
from sklearn.metrics.pairwise   import euclidean_distances
from sklearn.metrics.pairwise   import chi2_kernel
from sklearn.metrics.pairwise   import cosine_distances
from sklearn.metrics.pairwise   import manhattan_distances

### Euclidean distances

In [120]:
distance_matrix = pd.DataFrame(euclidean_distances(df, df), 
                         index = labels, 
                         columns = labels)
distance_matrix.to_csv(path + 'output/data_euclidean.csv')

### Cosine distance

In [121]:
distance_matrix = pd.DataFrame(euclidean_distances(df, df), 
                         index = labels, 
                         columns = labels)
distance_matrix.to_csv(path + 'output/data_cosine.csv')

### Manhattan distances

In [122]:
distance_matrix = pd.DataFrame(manhattan_distances(df, df), 
                         index = labels, 
                         columns = labels)
distance_matrix.to_csv(path + 'output/data_manhattan.csv')

Test sample

In [123]:
# Sample with different data
data = pd.read_csv(path + 'input/test.csv')
data = data.set_index('Product')
data = data.fillna(value = 0, axis = 1)
labels = list(data.index.values)
binary = ['vienguba', 'dviguba', 'pagrovimo_liftas', 'br1_desinepuse', 
          'br1_kairepuse', 'br1_grioveliofrezavimas', 'br1_kalibravimas', 
          'br1_pasukimostalas', 'br2_desinepuse', 'br2_kairepuse', 'br2_grioveliofrezavimas', 
          'br2_kalibravimas', 'briauna_r15', 'birauna_r70', 'briauna_kt', 'pasukimo_stalas_2', 
          'dalinimas1', 'Papildomas stalas']
data[data.columns.difference(binary)] = data[data.columns.difference(binary)].apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
data = data.fillna(value = 0, axis = 1)
data_ecld = pd.DataFrame(euclidean_distances(data, data), 
                         index = labels, 
                         columns = labels)
data_ecld.to_csv(path + 'output/data_ecld_test.csv')

## Klasterizavimo kriterijai

## Klasterizavimo metodai *(Clusterisation algorithms)*
* Gerai atskirti klasteriai *(Well-Separated Clusters)*;
* Centru pagrysti klusteriai *(Center-based clusters)*;
* Panašumu grysti clusteriai *(Contiguous clusters)*;
* Tankiu grysti klusteriai *(Density-based clusters)*;
* *(Property or conceptual)*;
* Klusteriai pagrysti objekto funkcija: *(Described by an an objective Function)*.
    * minimizuoja / maksimizuoja objekto funkciją;
    


### Padalijimo metodai *(Partitioning methods)*

#### K-Means Clustering

K-means is considered by many the gold standard when it comes to clustering due to its simplicity and performance, and it's the first one we'll try out. When you have no idea at all what algorithm to use, K-means is usually the first choice. Bear in mind that K-means might under-perform sometimes due to its concept: spherical clusters that are separable in a way so that the mean value converges towards the cluster center. To simply construct and train a K-means model, use the follow lines:

In [126]:
# K-means Clustering Model
KMeans = cluster.KMeans(n_clusters=kClusters, n_init=500)
# kmeans.fit(edgeMat)
KMeans.fit(df)

# Transform our data to list form and store them in results dict
results.update({'KMeans': list(KMeans.labels_)})

KMeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=500, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

### Hierarchinei metodai *(Hierarchical methids)*

#### Agglomerative Clustering

The main idea behind agglomerative clustering is that each node starts in its own cluster, and recursively merges with the pair of clusters that minimally increases a given linkage distance. The main advantage of agglomerative clustering (and hierarchical clustering in general) is that you don’t need to specify the number of clusters. That of course, comes with a price: performance. But, in scikit’s implementation, you can specify the number of clusters to assist the algorithm’s performance. To create and train an agglomerative model use the following code:

In [125]:
# Agglomerative Clustering Model
Agglomerative = cluster.AgglomerativeClustering(n_clusters=kClusters, linkage="ward")
Agglomerative.fit(df)

# Transform our data to list form and store them in results dict
results.update({'Agglomerative': list(Agglomerative.labels_)})

Agglomerative

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward',
            memory=Memory(cachedir=None), n_clusters=4,
            pooling_func=<function mean at 0x0000000004BA4D08>)

### Tankiu grysti metodai *(Density-Based methods)*

#### Gaussian Mixture

In [135]:
# Gaussian Mixture Clustering Model
GaussianMixture = mixture.GaussianMixture()
GaussianMixture.fit(df)

# Transform our data to list form and store them in results dict
#results.update({'GaussianMixture': list(GaussianMixture.labels_)})

GaussianMixture

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=1, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

### Tinkleliu grysti metodai *(Grid-Based methods)*

In [130]:
pd.DataFrame.from_dict(results)

,Product,agglomerative,dbscan,kmeans
0,BR3-IKW202,3,-1,2
1,BR3-ITW101-K,0,-1,1
2,BR3-ITW302,3,-1,0
3,BR3-ITW301,3,-1,0
4,BR3-ITW104,0,-1,1
5,BR3-ITW102,0,-1,1
6,BR3-ITW202,3,-1,2
7,BR3-ITW201,3,-1,2
8,BR3-IRW901,0,-1,1
9,BR3-IKW203,3,-1,2


## Klasterizavimo evoliucija *(Evaluation of clustering)*

## Tikrinimas / vertinimas rezultatų *(Validation of the results)*

### Clustering metrics
See the Clustering performance evaluation section of the user guide for further details.
There are two forms of evaluation:
* **supervised**, which uses a ground truth class values for each sample.
* **unsupervised**, which does not and measures the ‘quality’ of the model itself.

## Rezultatų interpretavimas *(Interpretation of the results)*

# SAMPLE

In [38]:
kClusters = 4
results = []
nmiResults = []
arsResults = []

G = nx.karate_club_graph()
groundTruth = [0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1]

In [39]:
def graphToEdgeMatrix(G):

    # Initialize Edge Matrix
    edgeMat = [[0 for x in range(len(G))] for y in range(len(G))]

    # For loop to set 0 or 1 ( diagonal elements are set to 1)
    for node in G:
        tempNeighList = G.neighbors(node)
        for neighbor in tempNeighList:
            edgeMat[node][neighbor] = 1
        edgeMat[node][node] = 1

    return edgeMat

In [40]:
edgeMat = graphToEdgeMatrix(G)

## K-Means Clustering

K-means is considered by many the gold standard when it comes to clustering due to its simplicity and performance, and it's the first one we'll try out. When you have no idea at all what algorithm to use, K-means is usually the first choice. Bear in mind that K-means might under-perform sometimes due to its concept: spherical clusters that are separable in a way so that the mean value converges towards the cluster center. To simply construct and train a K-means model, use the follow lines:

In [41]:
# K-means Clustering Model
kmeans = cluster.KMeans(n_clusters=kClusters, n_init=200)
# kmeans.fit(edgeMat)
kmeans.fit(data)

# Transform our data to list form and store them in results list
results.append(list(kmeans.labels_))

ValueError: could not convert string to float: 'chary'

#### Agglomerative Clustering

The main idea behind agglomerative clustering is that each node starts in its own cluster, and recursively merges with the pair of clusters that minimally increases a given linkage distance. The main advantage of agglomerative clustering (and hierarchical clustering in general) is that you don’t need to specify the number of clusters. That of course, comes with a price: performance. But, in scikit’s implementation, you can specify the number of clusters to assist the algorithm’s performance. To create and train an agglomerative model use the following code:

In [ ]:
# Agglomerative Clustering Model
agglomerative = cluster.AgglomerativeClustering(n_clusters=kClusters, linkage="ward")
agglomerative.fit(edgeMat)

# Transform our data to list form and store them in results list
results.append(list(agglomerative.labels_))

#### Spectral

The Spectral clustering technique applies clustering to a projection of the normalized Laplacian. When it comes to image clustering, spectral clustering works quite well. See the next few lines of Python for all the magic:

In [ ]:
# Spectral Clustering Model
spectral = cluster.SpectralClustering(n_clusters=kClusters, affinity="precomputed", n_init= 200)
spectral.fit(edgeMat)

# Transform our data to list form and store them in results list
results.append(list(spectral.labels_))

#### Affinity Propagation

Well this one is a bit different. Unlike the previous algorithms, you can see AF does not require the number of clusters to be determined before running the algorithm. AF, performs really well on several computer vision and biology problems, such as clustering pictures of human faces and identifying regulated transcripts:

In [ ]:
# Affinity Propagation Clustering Model
affinity = cluster.affinity_propagation(S=edgeMat, max_iter=200, damping=0.6)

# Transform our data to list form and store them in results list
results.append(list(affinity[1]))

# Metrics & Plotting